In [98]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import networkx.algorithms.community as nx_comm
%matplotlib inline

In [5]:
# read csv file into a pandas dataframe
df = pd.read_csv('wcc_graph.csv')

# # create Graph from dataframe
G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr=True, create_using=nx.Graph())

In [6]:
print("Total number of Nodes: " , G.number_of_nodes())
print("Total number of Edges: ", G.number_of_edges())

Total number of Nodes:  7126
Total number of Edges:  19743


In [11]:
# create a collection of louvain community dectectation algorithms iterations for optimum r value

collect_r = []
# no_of_Com = []
store_mod = []

r = 0
for j in range(20):
    for i in range(20):
        Community_Parts = nx_comm.louvain_communities(G , resolution = r)
        m = nx_comm.modularity(G, Community_Parts)
        collect_r.append(r)
        store_mod.append(m)
    r += 0.05

In [17]:
max_Mod = max(store_mod)

ind = 0
c = 0
for i in range(len(store_mod)):
    if(store_mod[i] == max_Mod):
        ind = i
        c += 1
print('index of r',ind)
print('no. of optimum values', c)
print('optimum value of r with highest Modularity', collect_r[ind],  " ",  max_Mod)

index of r 389
no. of optimum values 1
optimum value of r with highest Modularity 0.9500000000000003   0.5075116657138234


In [7]:
# create a collection of louvain community dectectation algorithms iterations

# r = collect_r[ind]
r = 0.9500000000000003
collect_results = []
store_modularity = []

for i in range(1000):
    Community_Parts = nx_comm.louvain_communities(G , resolution = r)
    m = nx_comm.modularity(G, Community_Parts)
    collect_results.append(Community_Parts)
    store_modularity.append(m)


In [8]:
max_Modularity = max(store_modularity)

index = 0
count = 0
for i in range(len(store_modularity)):
    if(store_modularity[i] == max_Modularity):
        index = i
        count += 1

In [9]:
final_ComDetect = collect_results[index]
avg = np.mean(store_modularity)

In [10]:
print("Maximum Modularity:" , max_Modularity)
print("Index of the max mod: ", index)
print("How many max mod exist:",count)
print("Number of partitions",len(final_ComDetect))

Maximum Modularity: 0.5111446318079998
Index of the max mod:  692
How many max mod exist: 1
Number of partitions 13


In [1]:
len(df)

NameError: name 'df' is not defined

In [11]:
# create a dictionary that maps nodes to community labels
node_to_comm = {n: i for i, comm in enumerate(final_ComDetect) for n in comm}

# create a pandas DataFrame from the dictionary
comm_df = pd.DataFrame.from_dict(node_to_comm, orient='index', columns=['new_Community'])
# add a column for node IDs


comm_df.index.name = 'source'
comm_df.reset_index(inplace=True)
comm_df



,source,new_Community
0,quavoy,0
1,lurkerfromstoneage,0
2,VivaBlasphemia,0
3,PDX_Cannabist,0
4,Celestial_creations,0
...,...,...
7121,AFurryThing23,12
7122,funatical,12
7123,kdmstr2,12
7124,DeliciousRazzmatazz,12


In [12]:
# create a pandas Series with the size of each community
comm_sizes = pd.Series([len(c) for c in final_ComDetect], name='Comm_size')

# create a new dataframe by merging comm_df and comm_sizes
comm_df_with_size = pd.merge(comm_df, comm_sizes, left_on='new_Community', right_index=True)

# print the resulting dataframe
print(comm_df_with_size['Comm_size'].max())

1530


In [13]:
df_merged = pd.merge(df, comm_df_with_size, on='source', how='left')

df_merged.to_csv('UnDir_edges_Comm.csv', index=False)


,source,target,Weight,post_id,post_title,post_score,comment_id,comment_score,post_no_of_comments,year,month,day,hour,minute,second,new_Community,Comm_size
0,ynotfoster,Sleight-Code,1,10y47g3,Finally got my driver's license again!!,78,j7w1hrz,6,2,2023,2,9,12,48,4,3,1530
1,ynotfoster,negamoonspoon,1,10y27vy,Spent first night unprepared for cold thunders...,17,j7w1d4j,2,6,2023,2,9,11,33,22,3,1530
2,ynotfoster,Im-skynet,2,10xfve7,I swear being honest in interviews gets you no...,44,j7sh4hx,9,39,2023,2,8,17,22,55,3,1530
3,ynotfoster,JayStrat,1,10w9007,"Duffel/tent suggestions, general suggestions",13,j7lv8t6,2,16,2023,2,7,11,22,44,3,1530
4,ynotfoster,0xca_,1,10qgckj,Quick question about Waffle House,100,j6q540o,6,43,2023,1,31,18,11,10,3,1530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19781,wlc,uhhang,1,eow6jb,Homeless Polllution Solution,0,fel2qdm,3,17,2020,1,14,19,49,23,9,1231
19782,iponeverything,uhhang,1,eow6jb,Homeless Polllution Solution,0,fejxg59,2,17,2020,1,14,19,49,23,9,1231
19783,LngDckStyle,TheCatanist,2,enxi93,Gym Membership. Is this a commonly known thing?,113,in2fko8,1,36,2020,1,12,19,8,59,9,1231
19784,ssnakeggirl,TheCatanist,2,enxi93,Gym Membership. Is this a commonly known thing?,113,feb6sna,2,36,2020,1,12,19,8,59,9,1231


In [14]:
# comm_df_with_size['Comm_size'].to_csv('UnDir_Comm.csv', index=False)

In [15]:
comm_df_with_size.to_csv('UnDir_Comm.csv', index=False)